In [21]:
import pandas as pd
from io import StringIO
import sys
import os
import csv
from sqlalchemy import create_engine,text
import datetime
# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.db_connection import DatabaseConnection
PATH = "../data/"




def read_data(file_name)-> pd.DataFrame:
    full_path = PATH + file_name + '.csv'
    row_data = pd.read_csv(full_path , quotechar='"' , low_memory=False)
    return row_data


def process_row_data(row_data: pd.DataFrame , file_date):

    row_data['ingested_at'] = file_date
    return row_data



def load_row_into_db(row_data: pd.DataFrame):
    
    
    db_connection = DatabaseConnection()

    row_data.columns = row_data.columns.str.lower()  # Fix columns first
    db_connection.load_dataframe_into_db(row_data ,"bronze" ,"covid")
    

    
def get_date_from_file_name(file_name:str) -> str:
    return file_name.split('.')[0]
    

In [22]:

def load_data_to_bronze(BASE):


    to_load_file_name = get_date_from_file_name(BASE)
    row_data = read_data(to_load_file_name)
    row_data = process_row_data(row_data , to_load_file_name)
    load_row_into_db(row_data)


In [23]:
load_data_to_bronze('01-02-2021.csv')